# Chapter 12 code snippets
This notebook contains all code snippets from chapter 12.

## Registering models

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
loans_ds = ws.datasets['loans']
experiment = Experiment(ws, "chapter-12-train")

In [ ]:
training_data, validation_data = loans_ds.random_split(
                             percentage = 0.8, seed=42)

X_train = training_data.drop_columns('approved_loan') \
            .to_pandas_dataframe()
y_train = training_data.keep_columns('approved_loan') \
            .to_pandas_dataframe().values.ravel()

X_validate = validation_data.drop_columns('approved_loan') \
                .to_pandas_dataframe()
y_validate = validation_data.keep_columns('approved_loan') \
                .to_pandas_dataframe().values.ravel()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

run = experiment.start_logging()

sk_model = LogisticRegression()
sk_model.fit(X_train, y_train)

y_predicted = sk_model.predict(X_validate)
accuracy = accuracy_score(y_validate, y_predicted)
print(accuracy)

run.log("accuracy", accuracy)

run.complete()

In [ ]:
import os
import joblib

os.makedirs('./model', exist_ok=True)
joblib.dump(value=sk_model,
            filename=
                os.path.join('./model/','model.joblib'))

In [ ]:
from sklearn import __version__ as sk_version
from azureml.core import Model

# run.upload_folder("model", "./model")
model = run.register_model(
        model_name="chapter12-loans",
        model_path="./model/",
        tags={ "accuracy": accuracy},
        properties={ "accuracy": accuracy},
        model_framework= Model.Framework.SCIKITLEARN,
        model_framework_version= sk_version,
        datasets=[("training", loans_ds)]
)


In [ ]:
from azureml.core import Model
offline_model = Model.register(
        ws,
        model_name="chapter12-pre-trained-loans",
        model_path="./model/",
        properties={ "accuracy": 0.828},
        model_framework= "ScikitLearn",
        model_framework_version= "0.22.2.post1"
)

In [ ]:
# This code snippet is from the next section
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name="chapter12-demanding-loans",
                       model_path="./model/",
                       model_framework=Model.Framework.SCIKITLEARN, 
                       model_framework_version="0.22.2.post1",
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1.5))

In [ ]:
from sklearn.linear_model import RidgeClassifier

new_model = RidgeClassifier(solver='svd')
new_model.fit(X_train, y_train)
y_predicted = new_model.predict(X_validate)
accuracy = accuracy_score(y_validate, y_predicted)

registered_model = Model(ws, name="chapter12-loans")
r_version = registered_model.version
r_acc = float(registered_model.properties['accuracy'])
if accuracy > r_acc:
    print(f"New model has better accuracy {accuracy}")
else:
    print(f"Registered model with version {r_version}" \
           " has better accuracy {r_acc}")

In [ ]:
import shutil

shutil.rmtree('./model',ignore_errors=True)

## Deploying real time endpoints

In [ ]:
from azureml.core import Workspace, Model

ws = Workspace.from_config()
model = Model(ws, name="chapter12-loans")

In [ ]:
no_code_service = Model.deploy(ws, "no-code-loans", [model])
no_code_service.wait_for_deployment(show_output=True)

In [ ]:
print(no_code_service.scoring_uri)

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [2000, 2, 45],
        [2000, 9, 45]
    ],
    'method': 'predict' # 'predict_proba'
})

output = no_code_service.run(input_payload)

print(output)

In [ ]:
import json

# This payload will also work
input_payload = json.dumps({
    "data":[
        {
            "income": 2000,
            "credit_cards": 2,
            "age": 45
        }
    ],
    "method": "predict"
})

output = no_code_service.run(input_payload)

print(output)

In [ ]:
no_code_service.delete()

### Understanding the model deployment options

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 
import sklearn

myEnv= Environment(name="sklearn-inference")
myEnv.python.conda_dependencies = CondaDependencies()
myEnv.python.conda_dependencies.add_conda_package(f"scikit-learn=={sklearn.__version__}")
myEnv.python.conda_dependencies.add_pip_package("azureml-defaults>=1.0.45")

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory= "./script",
                      entry_script='score.py', environment=myEnv)

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, "aci-loans", [model], inference_config, deployment_config)

service.wait_for_deployment(show_output=True)

In [ ]:
service.delete()

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=1337)

service = Model.deploy(ws, "local-loans", [model], inference_config, deployment_config)

service.wait_for_deployment()
print("If you are running this notebook in a compute instance, visit the")
print("endpoint at https://compute-instance-name-1337.region.instances.azureml.ms/")

In [ ]:
service.delete()

### Profiling the model resource requirements

In [ ]:
loans_ds = ws.datasets['loans']
prof_df = loans_ds.drop_columns('approved_loan') \
                        .to_pandas_dataframe()

prof_df['sample_request'] = \
    "{'data':[[" + prof_df['income'].map(str) \
  + ","+ prof_df['credit_cards'].map(str) \
  + "," + prof_df['age'].map(str) + "]]}"

prof_df = prof_df[['sample_request']]
prof_df.head()

In [ ]:
from azureml.core import Dataset

dstore = ws.get_default_datastore()
loan_req_ds = Dataset.Tabular.register_pandas_dataframe(
    dataframe=prof_df,
    target=(dstore,"/samples/loans-requests"),
    name="loans-requests",
    description="Sample requests for the loans model")

In [ ]:
profile = Model.profile(ws,
            'chapter12-loan',
            [model],
            inference_config,
            input_dataset=loan_req_ds,
            cpu=2,
            memory_in_gb=1)
profile.wait_for_completion(True)
print(profile.get_details())

### Monitoring with Application insights

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=0.5, enable_app_insights= True)

service = Model.deploy(ws, "aci-loans", [model], inference_config, deployment_config)

service.wait_for_deployment(show_output=True)

In [ ]:
service.update(enable_app_insights=True)

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [2000, 2, 45],
        [2000, 9, 45]
    ]
})

for x in range(10):
   print(service.run(input_payload))

In [ ]:
service.delete()

### Integrating with third party applications

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

myEnv.python.conda_dependencies.add_pip_package("inference_schema>=1.1.0")
inference_config = InferenceConfig(source_directory= "./script",
                      entry_script='score_v2.py', environment=myEnv)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=0.5)
service = Model.deploy(ws, "aci-loans", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
import json

service = ws.webservices['aci-loans']

input_payload = json.dumps({
    "data":[
        {
            "income": 2000,
            "credit_cards": 2,
            "age": 45
        },
        {
            "income": 2000,
            "credit_cards": 9,
            "age": 45
        }
    ]
})
print(service.run(input_payload))

input_payload = json.dumps({
    'data': [
        [2000, 2, 45],
        [2000, 9, 45]
    ]
})
print(service.run(input_payload))

In [ ]:
service.delete()

## Creating a batch inference pipeline

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
loans_ds = ws.datasets['loans']
compute_target = ws.compute_targets['cpu-sm-cluster']

In [ ]:
from azureml.core import Dataset

loans_df = loans_ds.drop_columns('approved_loan').to_pandas_dataframe()

print(f"Original DataFrame's size {loans_df.memory_usage(deep=True).sum()}")

for x in range(10):
    loans_df = loans_df.append(loans_df)
    
print(f"Expanded DataFrame's size {loans_df.memory_usage(deep=True).sum()}")

dstore = ws.get_default_datastore()
pending_loans_ds = Dataset.Tabular.register_pandas_dataframe(
    dataframe=loans_df,
    target=(dstore,"/samples/pending-loans"),
    name="pending-loans",
    description="Pending loans to be processed")

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 
import sklearn

pEnv= Environment(name="sklearn-parallel")
pEnv.python.conda_dependencies = CondaDependencies()
pEnv.python.conda_dependencies.add_conda_package(f"scikit-learn=={sklearn.__version__}")
pEnv.python.conda_dependencies.add_pip_package("azureml-core")
pEnv.python.conda_dependencies.add_pip_package("azureml-dataset-runtime[pandas,fuse]")

In [ ]:
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    source_directory='pipeline_step',
    entry_script='tabular_batch.py',
    mini_batch_size='100Kb',
    error_threshold=-1,
    output_action='append_row',
    append_row_file_name="loans_outputs.txt",
    environment=pEnv,
    compute_target=compute_target, 
    node_count=1,
    process_count_per_node=2,
    run_invocation_timeout=600
)

In [ ]:
from azureml.data import OutputFileDatasetConfig

datastore = ws.get_default_datastore()
step_output = OutputFileDatasetConfig(
    name= "results_store",
    destination=(datastore, '/inferences/loans/'))

In [ ]:
from azureml.pipeline.steps import ParallelRunStep

parallel_step = ParallelRunStep(
    name='chapter12-parallel-loans',
    inputs=[pending_loans_ds.as_named_input('loans')],
    output=step_output,
    parallel_run_config=parallel_run_config,
    allow_reuse=False
)

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallel_step])

pipeline_run = Experiment(ws, 'chapter12-parallel-run').submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails

RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion(show_output=True)